In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  with open(fn, 'wb') as f:
    f.write(uploaded[fn])
  print('File uploaded successfully!')

Saving biden.txt to biden.txt
User uploaded file "biden.txt" with length 269842 bytes
File uploaded successfully!


In [ ]:
!ls

biden.txt  sample_data


In [ ]:
!pip install datasets evaluate transformers[sentencepiece] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
# Read the plain text file
file_path = 'biden.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Convert to a pandas DataFrame
data = {'text': [line.strip() for line in lines]}
df = pd.DataFrame(data)

# Create a Dataset object
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 3555
})

In [ ]:
checkpoint = 'adarksky/biden-gpt2'

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dataset
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True)
    tokens["labels"] = tokens["input_ids"].copy()  # Labels are the same as input_ids for language modeling
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

NameError: name 'dataset' is not defined

In [ ]:
# Load model for causal language modeling
model = AutoModelForCausalLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

Step,Training Loss
500,0.087400
1000,0.051500
1500,0.036200
2000,0.030000
2500,0.023100


TrainOutput(global_step=2667, training_loss=0.044222871954180926, metrics={'train_runtime': 3705.7001, 'train_samples_per_second': 2.878, 'train_steps_per_second': 0.72, 'total_flos': 5573359042560000.0, 'train_loss': 0.044222871954180926, 'epoch': 3.0})

In [ ]:
input_text = "Americans"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [ ]:
# Generate encoded text
with torch.no_grad():
    outputs = model.generate(input_ids, min_length=100, max_length=500, num_return_sequences=1, do_sample=True, temperature=0.7, top_k=50)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# outputs.shape

torch.Size([1, 104])

In [ ]:
print(generated_text)

Americans, I’ve been told I am too conservative. Too many corporations are making too much money. Too few Americans are getting paid the same amount as their national average. Too many corporations are making obscene amounts of money. Too many corporations are making incredible profits. Too many corporations are making incredible profits by shifting profits and jobs overseas. It’s called trickle-down economics. It’s called trickle-down economics. Because a lot of you can’t lead America with the same kinds of jobs that a union man or woman could lead America with.


In [ ]:
model_path = './biden-gpt2'
model.save_pretrained(model_path)

tokenizer_path = './biden-gpt2-tok'
tokenizer.save_pretrained(tokenizer_path)

('./biden-gpt2-tok/tokenizer_config.json',
 './biden-gpt2-tok/special_tokens_map.json',
 './biden-gpt2-tok/vocab.json',
 './biden-gpt2-tok/merges.txt',
 './biden-gpt2-tok/added_tokens.json',
 './biden-gpt2-tok/tokenizer.json')

In [ ]:
!ls

biden-gpt2  biden-gpt2-tok  biden.txt  results	sample_data
